## Automate cytoscape network visualization with Python

We first activate the enviroment where we installed `py4cytoscape`

```
conda create --name cytoscape
conda activate cytoscape
conda install pip
pip install py4cytoscape
conda install ipykernel
```

Then, start Cytoscape and run the code below.

In [2]:
import numpy as np
import pandas as pd

import py4cytoscape as p4c
p4c.cytoscape_ping()
p4c.cytoscape_version_info()

You are connected to Cytoscape!


{'apiVersion': 'v1',
 'cytoscapeVersion': '3.10.1',
 'automationAPIVersion': '1.11.0',
 'py4cytoscapeVersion': '1.11.0'}

Load TFs annotation

In [3]:
tfs_dat = pd.read_csv('annotation/Nematostella_DToL_FINAL.tsv', sep='\t')
tfs_dat

,gene,gene_name,common_name,og,pfam,gene_list
0,Nvec_vc1.1_XM_032374162.2,Szt2,NaN,NaN,NaN,NaN
1,Nvec_vc1.1_XM_001626535.3,Rxylt1,NaN,NaN,NaN,NaN
2,Nvec_vc1.1_XM_001626549.3,Med8,NaN,NaN,Med8,NaN
3,Nvec_vc1.1_XM_032374171.2,Utp23,NaN,NaN,Fcf1,NaN
4,Nvec_vc1.1_XM_001626548.3,Ssr4,NaN,NaN,TRAP-delta,NaN
...,...,...,...,...,...,...
20530,Nvec_vc1.1_XM_048723200.1,NaN,NaN,NaN,NaN,NaN
20531,Nvec_vc1.1_XM_048723201.1,NaN,NaN,NaN,RVT_1,NaN
20532,Nvec_vc1.1_XM_048723203.1,Wdr45b,NaN,NaN,NaN,NaN
20533,Nvec_vc1.1_XM_048723202.1,NaN,NaN,NaN,F5_F8_type_C,NaN


Load GRN table

In [5]:
grn_dir = 'results/GRN/networks/'
grn_dat = pd.read_csv(
    grn_dir + 'grn_genes_expression_fc_0.4_accessibility_access_0.4_chromvar_4.tsv.gz',
    sep = '\t'
)
grn_dat = grn_dat.loc[~grn_dat['target_gene'].isna(), :]

Cell types

In [6]:
cell_types = grn_dat['cell_type'].unique()
cell_types

array(['cnidocyte', 'cnidocyte_gastrula', 'ecto_pharynx', 'ectoderm',
       'ecto_aboral', 'EMS', 'EMS_ecto_boundary',
       'gastro_circular_muscle_1', 'gastro_circular_muscle_2',
       'gastro_parietal_muscle', 'gastro_IRF1_2', 'gastro_somatic_gonad',
       'muscle_tentacle_retractor', 'muscle_mesentery_retractor',
       'digestive_filaments_1', 'digestive_filaments_2',
       'digestive_filaments_3', 'epidermis_1', 'epidermis_2',
       'precursors_PGC', 'precursors_endoNPC', 'precursors_NPC', 'NPC_1',
       'NPC_2', 'neuron_GATA_Islet_1', 'neuron_GATA_Islet_2',
       'neuron_Pou4_FoxL2_1', 'neuron_Pou4_FoxL2_2',
       'neuron_Pou4_FoxL2_3', 'neuronal_gastrula', 'gland', 'gland_mucin'],
      dtype=object)

For every cell type, we filter GRN data by gene expression, and generate networks.

In [7]:
for ct in cell_types:

    print(ct)
    
    # subset by cell type
    grn_cts = grn_dat[grn_dat['cell_type'] == ct]
    
    # filter genes by expression
    grn_cts = grn_cts.loc[grn_cts['expression_fc'] > 1.5, :]
    grn_cts = grn_cts.loc[grn_cts['target_expression_fc'] > 1.5, :]

    # set limits for expression: max 6
    grn_cts['expression_fc'] = grn_cts['expression_fc'].clip(upper=6)
    grn_cts['target_expression_fc'] = grn_cts['target_expression_fc'].clip(upper=6)

    # use common_name for gene name when available
    grn_cts.loc[grn_cts['common_name'].notna(), 'gene_name'] = grn_cts.loc[grn_cts['common_name'].notna(), 'common_name']
    grn_cts.loc[grn_cts['target_common_name'].notna(), 'target_gene_name'] = grn_cts.loc[grn_cts['target_common_name'].notna(), 'target_common_name']

    # use og tf family for tf gene name
    grn_cts.loc[grn_cts['gene_name'].isna(), 'gene_name'] = grn_cts.loc[grn_cts['gene_name'].isna(), 'og'].str.split('.').str[0]
    grn_cts.loc[grn_cts['target_gene_name'].isna(), 'target_gene_name'] = grn_cts.loc[grn_cts['target_gene_name'].isna(), 'target_og'].str.split('.').str[0]

    # if not a TF, set gene name to empty string
    grn_cts.loc[~grn_cts['gene'].isin(tfs_dat['gene']), 'gene_name'] = ''
    grn_cts = grn_cts.replace(np.nan, '', regex=True)

    # NODES

    # get unique source gene values in the network
    nodes_source = grn_cts.loc[:, ['gene', 'gene_name', 'expression_fc', 'zscore']]
    nodes_source.rename(columns={'gene': 'id', 'gene_name': 'name'}, inplace=True)
    nodes_source.drop_duplicates(ignore_index=True, inplace=True)
    nodes_source['type'] = 'source'

    # get unique target gene values in the network
    nodes_target = grn_cts.loc[:, ['target_gene', 'target_gene_name', 'target_expression_fc']]
    nodes_target.rename(columns={'target_gene': 'id', 'target_gene_name': 'name', 'target_expression_fc': 'expression_fc'}, inplace=True)
    nodes_target.drop_duplicates(ignore_index=True, inplace=True)
    nodes_target['zscore'] = 1e-5
    nodes_target['expression_fc'] = 1
    nodes_target['type'] = 'target'

    # get nodes_target that are not in nodes_source
    nodes_target = nodes_target.loc[~nodes_target['id'].isin(nodes_source['id'])]

    # combine nodes
    nodes = pd.concat([nodes_source, nodes_target], ignore_index=True)

    # change to str
    nodes['id'] = nodes['id'].astype(str)
    nodes['name'] = nodes['name'].astype(str)

    # EDGES

    # get edges
    edges = grn_cts.loc[:, ['gene', 'target_gene', 'in_silico_chip_score']]
    edges.rename(columns={'gene': 'source', 'target_gene': 'target'}, inplace=True)
    edges.drop_duplicates(ignore_index=True, inplace=True)

    # change to str
    edges['source'] = edges['source'].astype(str)
    edges['target'] = edges['target'].astype(str)

    # check that all edges sources are in node data.frame
    source_nodes = edges['source'].isin(nodes['id'])
    target_nodes = edges['target'].isin(nodes['id'])

    if source_nodes.all():
        print('All source nodes are in the node data.frame')
    else: 
        print(f'{sum(source_nodes == False)} target nodes are not in the node data.frame')

    if target_nodes.all():
        print('All target nodes are in the node data.frame')
    else: 
        print(f'{sum(target_nodes == False)} target nodes are not in the node data.frame')
        
    # NETWORK

    p4c.create_network_from_data_frames(nodes, edges, title=ct, collection='GRN')

    # style
    style_name = "grnStyle"
    defaults = {
        'NODE_SIZE': 1,
        'NODE_FILL_COLOR': '#D3D3D3',
        'EDGE_TRANSPARENCY': 120,
        'NODE_LABEL_POSITION': "W,E,c,0.00,0.00",
        'NODE_LABEL_FONT_SIZE': 28,
        'ARROW_SHAPE': "OPEN_DELTA",
        'ARROW_COLOR': '#D3D3D3',
        'ARROW_SIZE': 0.5
    }
    nodeLabels = p4c.map_visual_property('node label', 'name', 'p')
    edgeWidth = p4c.map_visual_property('edge width', 'in_silico_chip_score', 'p')
    p4c.create_visual_style(style_name, defaults, [nodeLabels, edgeWidth])
    p4c.set_visual_style(style_name)

    from py4cytoscape import gen_node_color_map
    from py4cytoscape import gen_node_size_map
    from py4cytoscape import gen_edge_arrow_map

    # source nodes
    p4c.set_node_color_mapping(**gen_node_color_map('expression_fc', p4c.palette_color_brewer_s_YlOrRd(), style_name=style_name))
    p4c.set_node_size_mapping(**gen_node_size_map('zscore', style_name=style_name))
    p4c.set_node_font_size_default(32, style_name=style_name)

    # target nodes
    target_nodes_ids = nodes.loc[nodes['type'] == 'target', 'name'].to_list()
    p4c.set_node_color_bypass(target_nodes_ids, '#D3D3D3')
    p4c.set_node_size_bypass(target_nodes_ids, 5)
    p4c.set_node_label_bypass(target_nodes_ids, '')

    # edges
    p4c.set_edge_target_arrow_shape_mapping(**gen_edge_arrow_map('interaction', style_name=style_name))

    # layout
    p4c.layout_network('kamada-kawai')
    p4c.bundle_edges()
    #p4c.clear_edge_bends()

    # show image
    p4c.notebook_export_show_image()


cnidocyte
All source nodes are in the node data.frame
All target nodes are in the node data.frame
Applying default style...
Applying preferred layout
cnidocyte_gastrula
All source nodes are in the node data.frame
All target nodes are in the node data.frame
Applying default style...
Applying preferred layout
ecto_pharynx
All source nodes are in the node data.frame
All target nodes are in the node data.frame
Applying default style...
Applying preferred layout
ectoderm
All source nodes are in the node data.frame
All target nodes are in the node data.frame
Applying default style...
Applying preferred layout
ecto_aboral
All source nodes are in the node data.frame
All target nodes are in the node data.frame
Applying default style...
Applying preferred layout
EMS
All source nodes are in the node data.frame
All target nodes are in the node data.frame
Applying default style...
Applying preferred layout
EMS_ecto_boundary
All source nodes are in the node data.frame
All target nodes are in the nod

Save session

In [8]:
p4c.session.save_session(grn_dir + 'cytoscape_session_expression_fc_1.5.cys')

{}